In [1]:
!pip install -q tensorflow_model_optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 2.2 MB/s eta 0:00:00


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import librosa
import librosa.display

from tensorflow.keras import layers, models
import tensorflow as tf

import tempfile
import zipfile
import time

from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils.layer_utils import count_params
# TF Optimization
import tensorflow_model_optimization as tfmot
from tensorflow_model_optimization.sparsity.keras import prune_low_magnitude

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
folder_path = "/content/drive/My Drive/donateacry_corpus_cleaned_and_updated_data/"
folder_crycelab2023 = "/content/drive/My Drive/CryCeLab2023/"
folder_ESC50 = "/content/drive/My Drive/ESC-50/"

Mounted at /content/drive


In [5]:
def wav_to_spectrogram(file_path, output_shape=(128, 64)):
    # Load the WAV file
    y, sr = librosa.load(file_path, sr=8000, dtype=np.float32)
    # print(sr)

    # Generate the spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    tmp = spectrogram.shape
    # Resize the spectrogram so that it can be fed into the model
    spectrogram = librosa.util.fix_length(spectrogram, size=output_shape[1], mode='constant', constant_values=0)

    # Normalize the spectrogram to values between 0 and 1, this really helps improve model accuracy
    spectrogram = librosa.util.normalize(spectrogram)

    return spectrogram, sr, y, tmp




# **457 Cleaned data**

In [6]:
shape = []
# Define the folder containing the subfolders with WAV files
data_folder = folder_path
# Define the labels (subfolder names)
labels = ['belly_pain', 'burping', 'discomfort', 'hungry', 'tired']

# Initialize empty lists to store spectrograms and labels
X0 = []
y0_model1 = []
y0_model2 = []

# Iterate through the labels
for label_idx, label in enumerate(labels):
    label_folder = os.path.join(data_folder, label)

    # Iterate through WAV files in the label folder
    for wav_file in os.listdir(label_folder):
        if wav_file.endswith('.wav'):
            wav_file_path = os.path.join(label_folder, wav_file)

            # Convert WAV to spectrogram using the wav_to_spectrogram function
            spectrogram, sr, au, tmp = wav_to_spectrogram(wav_file_path)

            shape.append(tmp[1])
            # Append the spectrogram to X_train
            X0.append(spectrogram)

            # Append the label index (numeric representation of the label) to y_train
            y0_model1.append(1)
            y0_model2.append(label_idx)

# Convert lists to NumPy arrays for further processing
X = np.array(X0)
y0_model1 = np.array(y0_model1)
y0_model2 = np.array(y0_model2)

# Check the shapes
print(f'X shape: {X.shape}')
print(f'y shape: {y0_model1.shape}')

X shape: (457, 128, 64)
y shape: (457,)


In [ ]:
# min(shape),shape

# **~1000 all baby cry data**

In [7]:
data_folder1 = folder_path + 'All Data'
label_old = {'hu': 0, 'bu': 0, 'bp': 0, 'dc': 0, 'ti': 0, 'lo': 0, 'ch': 0, 'sc': 0, 'dk': 0}

# Initialize empty lists to store spectrograms and labels
X1 = []
y1_model1, y1_model2 = [], []
for wav_file in os.listdir(data_folder1):
        if wav_file.endswith('.3gp') or wav_file.endswith('.caf'):
            wav_file_path = os.path.join(data_folder1, wav_file)
            # lb = wav_file_path[]
            if wav_file[-6:-4] not in label_old.keys(): continue
            label_old[wav_file[-6:-4]] += 1
            # Convert WAV to spectrogram using the wav_to_spectrogram function
            spectrogram, sr, au, tmp = wav_to_spectrogram(wav_file_path)

            # Append the spectrogram to X_train
            X1.append(spectrogram)

            # Append the label index (numeric representation of the label) to y_train
            y1_model1.append(1)

            label_idx = 0
            if wav_file[-6:-4] == 'bp': label_idx = 0
            elif wav_file[-6:-4] == "bu": label_idx = 1
            elif wav_file[-6:-4] == "dc": label_idx = 2
            elif wav_file[-6:-4] == "hu": label_idx = 3
            elif wav_file[-6:-4] == "ti": label_idx = 4

            y1_model2.append(label_idx)

# Convert lists to NumPy arrays for further processing
X1 = np.array(X1)
y1_model1 = np.array(y1_model1)
y1_model2 = np.array(y1_model2)
# Check the shapes
print(f'X shape: {X1.shape}')
print(f'y shape: {y1_model2.shape}')

X shape: (1126, 128, 64)
y shape: (1126,)


In [8]:
#check duplicates

old_files = os.listdir(data_folder1)

clean_files = []
for label_idx, label in enumerate(labels):
    label_folder = os.path.join(data_folder, label)
    # Iterate through WAV files in the label folder
    clean_files.extend(os.listdir(label_folder))


set(old_files)&set(clean_files)
'7b23d26e-6f97-4e8b-b520-a9fdde21ee91-1432275312750-1.7-m-48-hu.wav' in old_files

False

# **~20k cryceleb2023**

In [9]:
data_folder2 = folder_crycelab2023 + 'audio/dev/'
limit = 2000
# Initialize empty lists to store spectrograms and labels
X2 = []
y2_model1 = []
count = 0
for subfolder in os.listdir(data_folder2):
  if os.path.isdir(os.path.join(data_folder2,subfolder)):
    for subsubfolder in os.listdir(os.path.join(data_folder2,subfolder)):
      if os.path.isdir(os.path.join(data_folder2,subfolder, subsubfolder)):
        for wav_file in os.listdir(os.path.join(data_folder2,subfolder,subsubfolder)):
          if wav_file.endswith('.wav'):
              wav_file_path = os.path.join(data_folder2,subfolder,subsubfolder, wav_file)
              # Convert WAV to spectrogram using the wav_to_spectrogram function
              spectrogram, sr, au, tmp = wav_to_spectrogram(wav_file_path)
              shape.append(tmp[1])
              # Append the spectrogram to X_train
              X2.append(spectrogram)

              # Append the label index (numeric representation of the label) to y_train
              y2_model1.append(1)
              count += 1
          if count == limit:
            break
      if count == limit:
        break
  if count == limit:
    break

# Convert lists to NumPy arrays for further processing
X2 = np.array(X2)
y2_model1 = np.array(y2_model1)

# Check the shapes
print(f'X shape: {X2.shape}')
print(f'y shape: {y2_model1.shape}')

X shape: (1614, 128, 64)
y shape: (1614,)


In [ ]:
X2.shape

(1614, 128, 64)

In [10]:
label_esc50 = pd.read_csv(folder_ESC50 + 'ESC-50-master/meta/esc50.csv')
label_esc50

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,5-263831-B-6.wav,5,6,hen,False,263831,B
1996,5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,5-61635-A-8.wav,5,8,sheep,False,61635,A


In [11]:
esc50_cry_files = label_esc50.loc[label_esc50['category'] == 'crying_baby', 'filename'].values; esc50_cry_files

array(['1-187207-A-20.wav', '1-211527-A-20.wav', '1-211527-B-20.wav',
       '1-211527-C-20.wav', '1-22694-A-20.wav', '1-22694-B-20.wav',
       '1-60997-A-20.wav', '1-60997-B-20.wav', '2-107351-A-20.wav',
       '2-107351-B-20.wav', '2-151079-A-20.wav', '2-50665-A-20.wav',
       '2-50666-A-20.wav', '2-66637-A-20.wav', '2-66637-B-20.wav',
       '2-80482-A-20.wav', '3-151080-A-20.wav', '3-151081-A-20.wav',
       '3-151081-B-20.wav', '3-152007-A-20.wav', '3-152007-B-20.wav',
       '3-152007-C-20.wav', '3-152007-D-20.wav', '3-152007-E-20.wav',
       '4-167077-A-20.wav', '4-167077-B-20.wav', '4-167077-C-20.wav',
       '4-185575-A-20.wav', '4-185575-B-20.wav', '4-185575-C-20.wav',
       '4-59579-A-20.wav', '4-59579-B-20.wav', '5-151085-A-20.wav',
       '5-198411-A-20.wav', '5-198411-B-20.wav', '5-198411-C-20.wav',
       '5-198411-D-20.wav', '5-198411-E-20.wav', '5-198411-F-20.wav',
       '5-198411-G-20.wav'], dtype=object)

In [12]:
data_folder3 = folder_ESC50 + 'ESC-50-master/audio'

# Initialize empty lists to store spectrograms and labels
X3 = []
y3_model1 = []
for wav_file in os.listdir(data_folder3):
        if wav_file.endswith('.wav'):
            wav_file_path = os.path.join(data_folder3, wav_file)

            # Convert WAV to spectrogram using the wav_to_spectrogram function
            spectrogram, sr, au, tmp = wav_to_spectrogram(wav_file_path)

            shape.append(tmp[1])
            # Append the spectrogram to X_train
            X3.append(spectrogram)

            # Append the label index (numeric representation of the label) to y_train
            if wav_file in esc50_cry_files: y3_model1.append(1)
            else: y3_model1.append(0)

# Convert lists to NumPy arrays for further processing
X3 = np.array(X3)
y3_model1 = np.array(y3_model1)

# Check the shapes
print(f'X shape: {X3.shape}')
print(f'y shape: {y3_model1.shape}')

X shape: (2000, 128, 64)
y shape: (2000,)


In [ ]:
sum(y3_model1 == 1)

40


# **CREMA-D**

In [13]:
import os
import subprocess

# Define the GitHub repository URL
github_repo_url = "https://github.com/CheyneyComputerScience/CREMA-D.git"

# Clone the repository
subprocess.run(["git", "clone", github_repo_url])

# Define the path to the cloned repository
repo_path = "CREMA-D"

In [14]:
X4 = []
y4_model1 = []

# Initialize empty lists to store WAV file paths and folder names
wav_files = []
folder_names = []

# Walk through the directory structure to find WAV files and their folders
for root, dirs, files in os.walk(repo_path):
    for file in files:
        # if file.endswith(".wav"):
        if file.endswith(".wav"):
            # Get the full path of the WAV file
            wav_path = os.path.join(root, file)

            # Extract the folder name from the path
            folder_name = os.path.basename(root)

            # Append the WAV file path and folder name to the respective lists
            wav_files.append(wav_path)

            spectrogram, sr, au, tmp = wav_to_spectrogram(wav_path)

            shape.append(tmp[1])
            # Append the spectrogram to X_train
            X4.append(spectrogram)

            # Append the label index (numeric representation of the label) to y_train
            y4_model1.append(0)

# Convert lists to NumPy arrays for further processing
X4 = np.array(X4)
y4_model1 = np.array(y4_model1)

# Check the shapes
print(f'X shape: {X4.shape}')
print(f'y shape: {y4_model1.shape}')

X shape: (7442, 128, 64)
y shape: (7442,)


# **Concat data**


In [15]:
x_final_model1 = np.concatenate((X0, X1, X2,X3, X4), axis  = 0)
# x_final_model1 = np.concatenate((X0, X2, X3, X4), axis  = 0)
x_final_model1.shape

(12639, 128, 64)

In [16]:
y_final_model1 = np.concatenate((y0_model1, y1_model1, y2_model1,y3_model1, y4_model1), axis  = 0)
# y_final_model1 = np.concatenate((y0_model1, y2_model1, y3_model1, y4_model1), axis  = 0)
y_final_model1.shape

(12639,)

In [17]:
x_final_model2 = np.concatenate((X0, X1), axis  = 0)
x_final_model2.shape

(1583, 128, 64)

In [18]:
y_final_model2 = np.concatenate((y0_model2, y1_model2), axis  = 0)
y_final_model2.shape

(1583,)

In [19]:
np.save('y_final_model2.npy', y_final_model2)
np.save('x_final_model2.npy', x_final_model2)

In [21]:
y_final_model1, y_final_model2

(array([1, 1, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 3, 0, 3]))

In [20]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(x_final_model1, y_final_model1, test_size=0.10, random_state=215)

In [ ]:
!ls

CREMA-D  drive	qkeras	sample_data  x_final_model2.npy  y_final_model2.npy


In [ ]:
folder_out = "/content/drive/My Drive/Xy_data/"

In [ ]:
np.save('X_model1.npy',  x_final_model1)

In [ ]:
np.save('./X_model2.npy',  x_final_model2)

In [ ]:
np.save('./y_model1.npy',  y_final_model1)

In [ ]:
np.save('./y_model2.npy',  y_final_model2)

In [22]:
# Create the model
num_classes = len(np.unique(y_train))
x_shape = X[0].shape

model = models.Sequential([
    layers.Input(shape=(x_shape[0], x_shape[1])),  # Adjust the input shape based on your spectrogram size
    layers.Reshape(target_shape=(x_shape[0], x_shape[1], 1)),  # Add a channel dimension
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(x_shape[0], activation='relu'),
    layers.Dense(num_classes, activation='softmax') #sigmoid
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [23]:
# Convert labels to one-hot encoding (assuming you have more than one class)
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes)
y_test_encoded = tf.keras.utils.to_categorical(y_test, num_classes)

# Train the model
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_encoded, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test accuracy: {accuracy:.2f}')

Epoch 1/10
320/320 [==============================] - 92s 282ms/step - loss: 0.1201 - accuracy: 0.9493 - val_loss: 0.0897 - val_accuracy: 0.9640
Epoch 2/10
320/320 [==============================] - 86s 269ms/step - loss: 0.0480 - accuracy: 0.9819 - val_loss: 0.1045 - val_accuracy: 0.9649
Epoch 3/10
320/320 [==============================] - 77s 242ms/step - loss: 0.0163 - accuracy: 0.9944 - val_loss: 0.0786 - val_accuracy: 0.9684
Epoch 4/10
320/320 [==============================] - 85s 267ms/step - loss: 0.0044 - accuracy: 0.9991 - val_loss: 0.1495 - val_accuracy: 0.9649
Epoch 5/10
320/320 [==============================] - 83s 259ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.1484 - val_accuracy: 0.9675
Epoch 6/10
320/320 [==============================] - 79s 248ms/step - loss: 6.0180e-04 - accuracy: 0.9998 - val_loss: 0.1385 - val_accuracy: 0.9692
Epoch 7/10
320/320 [==============================] - 86s 268ms/step - loss: 3.8631e-04 - accuracy: 0.9999 - val_loss: 0.1542 

## Model Pruning

### Analyze Model weights and size

In [24]:
def check_model_weights(model):
  for i, w in enumerate(model.get_weights()):
    print(model.weights[i].name,"Total:",w.size, "Zeros:", round(np.sum(np.abs(w) <= 0.0001) / w.size * 100,6),"%")

check_model_weights(model)

conv2d/kernel:0 Total: 288 Zeros: 0.0 %
conv2d/bias:0 Total: 32 Zeros: 0.0 %
dense/kernel:0 Total: 7999488 Zeros: 0.46963 %
dense/bias:0 Total: 128 Zeros: 0.0 %
dense_1/kernel:0 Total: 256 Zeros: 0.0 %
dense_1/bias:0 Total: 2 Zeros: 0.0 %


In [25]:
def compare_model_sizes(model):
  _, model_file = tempfile.mkstemp(".h5")
  tf.keras.models.save_model(model, model_file, include_optimizer=False)
  # Zip the .h5 model file
  _, zip3 = tempfile.mkstemp(".zip")
  with zipfile.ZipFile(zip3, "w", compression=zipfile.ZIP_DEFLATED) as f:
      f.write(model_file)
  print("Model before zip: %.2f Kb"% (os.path.getsize(model_file) / float(1000)))
  print("Model after zip: %.2f Kb"% (os.path.getsize(zip3) / float(1000)))

compare_model_sizes(model)

Model before zip: 32020.42 Kb
Model after zip: 29800.67 Kb


### Model Prune

In [26]:
# Define model for pruning
epochs = 10
end_step = np.ceil(len(X_train) / batch_size).astype(np.int32) * epochs

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# Optimizer
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
# Compile
model_for_pruning.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

# Callback
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep()
]

# Train
start_time = time.time()

history = model_for_pruning.fit(X_train, y_train_encoded, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks, verbose=1)

execution_time = (time.time() - start_time)/60.0
print("Training execution time (mins)",execution_time)

# Evaluate the model on the test set
y_pred = model_for_pruning.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test_encoded, axis=1)

accuracy = accuracy_score(y_test_classes, y_pred_classes)
print(f'Test accuracy: {accuracy:.2f}')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 128, 64, 1)        1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 126, 62, 32)       610       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 63, 31, 32)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_flatte  (None, 62496)             1         
 n (PruneLowMagnitude)                                           
                                                        

### Analyze Pruned Weights

In [27]:
# Get the model back after pruning
after_prune = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
after_prune.summary()

check_model_weights(after_prune)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 128, 64, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 126, 62, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 31, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 62496)             0         
                                                                 
 dense (Dense)               (None, 128)               7999616   
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                        

### Model Size after Pruning

In [28]:
compare_model_sizes(after_prune)

Model before zip: 32020.42 Kb
Model after zip: 9467.03 Kb


## Custom Tests

In [29]:
# Try Customized Test
# Test labels: [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]

folder_test = "/content/drive/My Drive/custom_test/"

In [30]:
# Initialize empty lists to store spectrograms and labels
Xtest = []
ytest = []
filename = []
for wav_file in os.listdir(folder_test):
        # print(wav_file)
        if wav_file.endswith('.wav'):
            wav_file_path = os.path.join(folder_test, wav_file)
            spectrogram, sr, au, tmp = wav_to_spectrogram(wav_file_path)

            # Append the spectrogram to X_train
            Xtest.append(spectrogram)
            filename.append(wav_file)
            # Append the label index (numeric representation of the label) to y_train
            ytest.append(1)

# Convert lists to NumPy arrays for further processing
Xtest = np.array(Xtest)
ytest = np.array(ytest)

# Check the shapes
print(f'X shape: {Xtest.shape}')
print(f'y shape: {ytest.shape}')

X shape: (11, 128, 64)
y shape: (11,)


In [32]:
# True labels: [0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1]
ypred = model.predict(Xtest)
ypred_classes = np.argmax(ypred, axis=1)
ypred_classes, ypred

1/1 [==============================] - 0s 44ms/step


(array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]),
 array([[9.9999994e-01, 2.8658986e-17],
        [9.9999994e-01, 1.2890738e-10],
        [9.9999994e-01, 8.9935709e-23],
        [1.0907292e-01, 8.9092714e-01],
        [6.5363580e-01, 3.4636414e-01],
        [9.9999994e-01, 9.3285035e-29],
        [9.9999994e-01, 5.9491697e-13],
        [9.9999994e-01, 4.5529572e-16],
        [1.0000000e+00, 7.3675659e-19],
        [1.0000000e+00, 3.6041526e-20],
        [2.1241959e-03, 9.9787581e-01]], dtype=float32))

In [33]:
ypred = model_for_pruning.predict(Xtest)
ypred_classes = np.argmax(ypred, axis=1)
ypred_classes, filename

1/1 [==============================] - 0s 67ms/step


(array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]),
 ['test 1.wav',
  'test 2.wav',
  'test 3.wav',
  'test 4.wav',
  'test 5.wav',
  'test 6.wav',
  'test 7.wav',
  'test 8.wav',
  'test 9.wav',
  'test 10.wav',
  'test 11.wav'])